In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import networkx.convert_matrix
import os.path as osp

import torch
from torch_geometric.data import Data

import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
from tqdm import tqdm
from torch_geometric.nn import SAGEConv

In [2]:
import math
import random
import os.path as osp
from itertools import chain

import numpy as np
from sklearn.metrics import roc_auc_score
from scipy.sparse.csgraph import shortest_path
import torch
import torch.nn.functional as F
from torch.nn import BCEWithLogitsLoss
from torch.nn import ModuleList, Linear, Conv1d, MaxPool1d

from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, global_sort_pool
from torch_geometric.data import Data, InMemoryDataset, DataLoader
from torch_geometric.utils import (negative_sampling, add_self_loops,
                                   train_test_split_edges, k_hop_subgraph,
                                   to_scipy_sparse_matrix)

In [3]:
def pre(train, content):
    
    
    content.rename(columns={ content.columns[0]: "id" }, inplace=True)
    contentArray = content.values
    contentArray_sort = contentArray[contentArray[:,0].argsort()]
    contentArray_sort = pd.DataFrame(contentArray_sort)

    unconnected_pairs = []
    matrix =np.zeros( (contentArray_sort.shape[0], contentArray_sort.shape[0]) )
    
    for index,row in train.iterrows():
        if row[3]==1:
            x = row[1]
            y = row[2]
            matrix[x, y] = 1
            matrix[y, x] = 1
        else:
            x = row[1]
            y = row[2]
            matrix[x, y] = 0
            matrix[y, x] = 0

    return np.asmatrix(matrix), contentArray_sort
def edge_sort_train(train):
    train_link = train.loc[:,['to', 'from', 'label']]

    for i in train_link.index:
        t = train_link.loc[i]['to']
        f = train_link.loc[i]['from']
        if t > f :
            train_link.loc[i]['to'] = f
            train_link.loc[i]['from'] = t

    train_link = train_link.sort_values(by='to')
    train_link = train_link[~train_link.duplicated()] # -> with duplicated edge exists
    train_link.index = range(train_link.shape[0])

    return train_link

def edge_sort_test(train):
    train_link = train.loc[:,['id', 'to', 'from']]

    for i in train_link.index:
        t = train_link.loc[i]['to']
        f = train_link.loc[i]['from']
        if t > f :
            train_link.loc[i]['to'] = f
            train_link.loc[i]['from'] = t

    train_link = train_link.sort_values(by='to')
    train_link.index = range(train_link.shape[0])

    return train_link

def get_fea(e_sort, str_fea):
    
    fea = []
    for i in e_sort.index:

        t = str_fea.loc[e_sort.loc[i, 'to'], :].values
        f = str_fea.loc[e_sort.loc[i, 'from'], :].values
        fea.append(t * f)

    fea = np.array(fea)
    
    return fea

In [4]:
edge_test = pd.read_csv('test_2.csv')
e_sort_test = edge_sort_test(edge_test)
u_ex = pd.read_csv('upload_2.csv')

train = pd.read_csv('train_2.csv')
edge_train = edge_sort_train(train)
content = pd.read_csv('content_2.csv', delimiter='\t',header=None)
content = content.drop(0, axis=1)
train_adj, content_p = pre(train, content)
n_fea = pd.read_csv('n_100fea2.csv')

# pd.read_csv('n_100fea2.csv')

edge_c = np.concatenate((edge_train.loc[:, ['from', 'to']].values, edge_train.loc[:, ['to', 'from']].values), axis=0)
edge_index = torch.tensor(edge_c, dtype=torch.long)

d_t2 = Data(edge_index=edge_index.t())
d_t2.num_nodes= content.shape[0]
d_t2.x = torch.tensor(n_fea.values, dtype=torch.float32)
# data = train_test_split_edges(data,  test_ratio=0.001)

<ipython-input-3-26aabe0fad8d>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_link.loc[i]['to'] = f
<ipython-input-3-26aabe0fad8d>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_link.loc[i]['from'] = t


In [5]:
class SEALDataset(InMemoryDataset):
    def __init__(self, dataset, num_hops, split='train'):
        self.data = dataset
        self.num_hops = num_hops
        super(SEALDataset, self).__init__('data\\dt2')
        index = ['train', 'val', 'test'].index(split)
        self.data, self.slices = torch.load(self.processed_paths[index])

    @property
    def processed_file_names(self):
        return ['SEAL_train_data.pt', 'SEAL_val_data.pt', 'SEAL_test_data.pt']

    def process(self):
        random.seed(12345)
        torch.manual_seed(12345)

        data = train_test_split_edges(self.data, 0.001)

        edge_index, _ = add_self_loops(data.train_pos_edge_index)
        data.train_neg_edge_index = negative_sampling(
            edge_index, num_nodes=data.num_nodes,
            num_neg_samples=data.train_pos_edge_index.size(1))

        self.__max_z__ = 0

        # Collect a list of subgraphs for training, validation and test.
        train_pos_list = self.extract_enclosing_subgraphs(
            data.train_pos_edge_index, data.train_pos_edge_index, 1)
        train_neg_list = self.extract_enclosing_subgraphs(
            data.train_neg_edge_index, data.train_pos_edge_index, 0)

        val_pos_list = self.extract_enclosing_subgraphs(
            data.val_pos_edge_index, data.train_pos_edge_index, 1)
        val_neg_list = self.extract_enclosing_subgraphs(
            data.val_neg_edge_index, data.train_pos_edge_index, 0)

        test_edge = torch.tensor(e_sort_test.loc[:,['to', 'from']].values.T, dtype=torch.int64).cuda()
        test_pos_list = self.extract_enclosing_subgraphs(
            test_edge, data.train_pos_edge_index, 1)
#         test_neg_list = self.extract_enclosing_subgraphs(
#             data.test_neg_edge_index, data.train_pos_edge_index, 0)

        # Convert labels to one-hot features.
        for data in chain(train_pos_list, train_neg_list, val_pos_list,
                          val_neg_list, test_pos_list):
            data.x = F.one_hot(data.z, self.__max_z__ + 1).to(torch.float)

        torch.save(self.collate(train_pos_list + train_neg_list),
                   self.processed_paths[0])
        torch.save(self.collate(val_pos_list + val_neg_list),
                   self.processed_paths[1])
        torch.save(self.collate(test_pos_list),
                   self.processed_paths[2])

    def extract_enclosing_subgraphs(self, link_index, edge_index, y):
        data_list = []
        for src, dst in link_index.t().tolist():
            sub_nodes, sub_edge_index, mapping, _ = k_hop_subgraph(
                [src, dst], self.num_hops, edge_index, relabel_nodes=True)
            src, dst = mapping.tolist()

            # Remove target link from the subgraph.
            mask1 = (sub_edge_index[0] != src) | (sub_edge_index[1] != dst)
            mask2 = (sub_edge_index[0] != dst) | (sub_edge_index[1] != src)
            sub_edge_index = sub_edge_index[:, mask1 & mask2]

            # Calculate node labeling.
            z = self.drnl_node_labeling(sub_edge_index, src, dst,
                                        num_nodes=sub_nodes.size(0))

            data = Data(x=self.data.x[sub_nodes], z=z,
                        edge_index=sub_edge_index, y=y)
            data_list.append(data)

        return data_list

    def drnl_node_labeling(self, edge_index, src, dst, num_nodes=None):
        # Double-radius node labeling (DRNL).
        src, dst = (dst, src) if src > dst else (src, dst)
        adj = to_scipy_sparse_matrix(edge_index, num_nodes=num_nodes).tocsr()

        idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
        adj_wo_src = adj[idx, :][:, idx]

        idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
        adj_wo_dst = adj[idx, :][:, idx]

        dist2src = shortest_path(adj_wo_dst, directed=False, unweighted=True,
                                 indices=src)
        dist2src = np.insert(dist2src, dst, 0, axis=0)
        dist2src = torch.from_numpy(dist2src)

        dist2dst = shortest_path(adj_wo_src, directed=False, unweighted=True,
                                 indices=dst - 1)
        dist2dst = np.insert(dist2dst, src, 0, axis=0)
        dist2dst = torch.from_numpy(dist2dst)

        dist = dist2src + dist2dst
        dist_over_2, dist_mod_2 = dist // 2, dist % 2

        z = 1 + torch.min(dist2src, dist2dst)
        z += dist_over_2 * (dist_over_2 + dist_mod_2 - 1)
        z[src] = 1.
        z[dst] = 1.
        z[torch.isnan(z)] = 0.

        self.__max_z__ = max(int(z.max()), self.__max_z__)

        return z.to(torch.long)

In [6]:

# dataset = Planetoid(root='data/Planetoid', name='Cora')


In [7]:
train_dataset = SEALDataset(d_t2, num_hops=2, split='train')
val_dataset = SEALDataset(d_t2, num_hops=2, split='val')
test_dataset = SEALDataset(d_t2, num_hops=2, split='test')

Processing...


ValueError: indices out of range 0...N

In [21]:
# train_dataset = SEALDataset(dataset, num_hops=2, split='train')
# val_dataset = SEALDataset(dataset, num_hops=2, split='val')
# test_dataset = SEALDataset(dataset, num_hops=2, split='test')

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512)
test_loader = DataLoader(test_dataset, batch_size=512)

In [22]:
class DGCNN(torch.nn.Module):
    def __init__(self, hidden_channels, num_layers, GNN=GCNConv, k=0.6):
        super(DGCNN, self).__init__()

        if k < 1:  # Transform percentile to number.
            num_nodes = sorted([data.num_nodes for data in train_dataset])
            k = num_nodes[int(math.ceil(k * len(num_nodes))) - 1]
            k = max(10, k)
        self.k = int(k)

        self.convs = ModuleList()
        self.convs.append(GNN(train_dataset.num_features, hidden_channels))
        for i in range(0, num_layers - 1):
            self.convs.append(GNN(hidden_channels, hidden_channels))
        self.convs.append(GNN(hidden_channels, 1))

        conv1d_channels = [16, 32]
        total_latent_dim = hidden_channels * num_layers + 1
        conv1d_kws = [total_latent_dim, 5]
        self.conv1 = Conv1d(1, conv1d_channels[0], conv1d_kws[0],
                            conv1d_kws[0])
        self.maxpool1d = MaxPool1d(2, 2)
        self.conv2 = Conv1d(conv1d_channels[0], conv1d_channels[1],
                            conv1d_kws[1], 1)
        dense_dim = int((self.k - 2) / 2 + 1)
        dense_dim = (dense_dim - conv1d_kws[1] + 1) * conv1d_channels[1]
        self.lin1 = Linear(dense_dim, 128)
        self.lin2 = Linear(128, 1)

    def forward(self, x, edge_index, batch):
        xs = [x]
        for conv in self.convs:
            xs += [torch.tanh(conv(xs[-1], edge_index))]
        x = torch.cat(xs[1:], dim=-1)

        # Global pooling.
        x = global_sort_pool(x, batch, self.k)
        x = x.unsqueeze(1)  # [num_graphs, 1, k * hidden]
        x = F.relu(self.conv1(x))
        x = self.maxpool1d(x)
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # [num_graphs, dense_dim]

        # MLP.
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCNN(hidden_channels=32, num_layers=3).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)


def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        logits = model(data.x, data.edge_index, data.batch)
        loss = BCEWithLogitsLoss()(logits.view(-1), data.y.to(torch.float))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs

    return total_loss / len(train_dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    y_pred, y_true = [], []
    for data in loader:
        data = data.to(device)
        logits = model(data.x, data.edge_index, data.batch)
        y_pred.append(logits.view(-1).cpu())
        y_true.append(data.y.view(-1).cpu().to(torch.float))

    return roc_auc_score(torch.cat(y_true), torch.cat(y_pred))

In [23]:
best_val_auc = test_auc = 0
for epoch in range(1, 51):
    loss = train()
    val_auc = test(val_loader)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        test_auc = test(test_loader)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

Epoch: 01, Loss: 0.6928, Val: 0.5662, Test: 0.5639
Epoch: 02, Loss: 0.6917, Val: 0.5683, Test: 0.5657
Epoch: 03, Loss: 0.6889, Val: 0.5711, Test: 0.5683
Epoch: 04, Loss: 0.6848, Val: 0.5752, Test: 0.5708
Epoch: 05, Loss: 0.6793, Val: 0.5838, Test: 0.5741
Epoch: 06, Loss: 0.6754, Val: 0.5884, Test: 0.5811
Epoch: 07, Loss: 0.6720, Val: 0.5941, Test: 0.5914
Epoch: 08, Loss: 0.6697, Val: 0.6020, Test: 0.5971
Epoch: 09, Loss: 0.6675, Val: 0.5952, Test: 0.5971
Epoch: 10, Loss: 0.6666, Val: 0.6005, Test: 0.5971
Epoch: 11, Loss: 0.6655, Val: 0.6001, Test: 0.5971
Epoch: 12, Loss: 0.6647, Val: 0.6039, Test: 0.6000
Epoch: 13, Loss: 0.6635, Val: 0.5926, Test: 0.6000
Epoch: 14, Loss: 0.6630, Val: 0.6032, Test: 0.6000
Epoch: 15, Loss: 0.6619, Val: 0.6045, Test: 0.6059
Epoch: 16, Loss: 0.6617, Val: 0.5967, Test: 0.6059
Epoch: 17, Loss: 0.6601, Val: 0.5944, Test: 0.6059
Epoch: 18, Loss: 0.6594, Val: 0.5932, Test: 0.6059
Epoch: 19, Loss: 0.6593, Val: 0.5956, Test: 0.6059
Epoch: 20, Loss: 0.6585, Val: 0